<a href="https://colab.research.google.com/github/koleshjr/Resume_Classification/blob/main/Resume_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resume Classifier

In [ ]:
!pip install -q datasets
!pip install transformers
!pip install optuna
!pip install sentencepiece
!pip install evaluate


!pip install mlflow wandb



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 19.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.8 MB/s eta 0:00:00
Looking in index

In [1]:

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pathlib import Path
path = Path('/content/drive/MyDrive/RESUME_YG')



In [3]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import seaborn as sns 
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import mlflow
import os
import seaborn as sns
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import transformers 
import wandb

In [4]:
!wandb login 
wandb.init(project="Resume_Classifier")

wandb: Currently logged in as: koleshjr (team-kolesh). Use `wandb login --relogin` to force relogin


wandb: Currently logged in as: koleshjr (team-kolesh). Use `wandb login --relogin` to force relogin


In [5]:
%env WANDB_LOG_MODEL=true   #log every trained model

env: WANDB_LOG_MODEL=true   #log every trained model


In [6]:

transformers.set_seed(1) 

# ds_1 = pd.read_csv(path + 'FA-KES-Dataset.csv')
df = pd.read_csv(path/'UpdatedResumeDataSet.csv')
df.head()
     






,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [7]:
print(df.Category.nunique())
df.Category.value_counts()



25


Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
Blockchain                   40
ETL Developer                40
Operations Manager           40
Data Science                 40
Sales                        40
Mechanical Engineer          40
Arts                         36
Database                     33
Electrical Engineering       30
Health and fitness           30
PMO                          30
Business Analyst             28
DotNet Developer             28
Automation Testing           26
Network Security Engineer    25
SAP Developer                24
Civil Engineer               24
Advocate                     20
Name: Category, dtype: int64

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encoded_labels = le.fit_transform(df['Category'])
df['Category']= le.fit_transform(df['Category'])
# generate dictionary mapping integer labels back to categories
class_dict = dict(zip(range(len(le.classes_)), le.classes_))
class_dict

{0: 'Advocate',
 1: 'Arts',
 2: 'Automation Testing',
 3: 'Blockchain',
 4: 'Business Analyst',
 5: 'Civil Engineer',
 6: 'Data Science',
 7: 'Database',
 8: 'DevOps Engineer',
 9: 'DotNet Developer',
 10: 'ETL Developer',
 11: 'Electrical Engineering',
 12: 'HR',
 13: 'Hadoop',
 14: 'Health and fitness',
 15: 'Java Developer',
 16: 'Mechanical Engineer',
 17: 'Network Security Engineer',
 18: 'Operations Manager',
 19: 'PMO',
 20: 'Python Developer',
 21: 'SAP Developer',
 22: 'Sales',
 23: 'Testing',
 24: 'Web Designing'}

### Training

In [10]:
model_nm = "distilbert-base-uncased"

tokz = AutoTokenizer.from_pretrained(model_nm, truncation=True, do_lower_case=True)



model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=25)#, gradient_checkpointing = True).to(device)
# gradient_checkpointing: for saving memory - but will make computation slower

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [11]:
# In the first step we will split the data in training and remaining dataset
df = df.rename(columns = {'Category':'label',
                          'Resume':'inputs'})
df_train, df_temp = train_test_split(df, train_size=0.8, stratify = df['label'])

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)

df_valid, df_test = train_test_split(df_temp, test_size=0.5,stratify = df_temp['label'])

df_train.shape, df_valid.shape, df_test.shape

((769, 2), (96, 2), (97, 2))

In [12]:
adverse = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "valid": Dataset.from_pandas(df_valid),
    "test": Dataset.from_pandas(df_test)
    })

Tokenizing function

In [13]:
adverse

DatasetDict({
    train: Dataset({
        features: ['label', 'inputs', '__index_level_0__'],
        num_rows: 769
    })
    valid: Dataset({
        features: ['label', 'inputs', '__index_level_0__'],
        num_rows: 96
    })
    test: Dataset({
        features: ['label', 'inputs', '__index_level_0__'],
        num_rows: 97
    })
})

In [14]:
def tokenize(x): return tokz(x["inputs"],truncation=True, padding=True, max_length=512)



adverse_encoded = adverse.map(tokenize, batched=True, batch_size=8,remove_columns=('__index_level_0__')) #, 


Map:   0%|          | 0/769 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [21]:
batch_size = 16
# the default logging_steps in TrainingArguments is set to 500 steps, so no loss is reported before 500 steps
model_name = f"{model_nm}-finetuned-resume_yg"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=10,
                                  learning_rate= 1e-05,
                                  per_device_train_batch_size=batch_size,
#                                   per_device_eval_batch_size=batch_size,
                                  weight_decay=0.03,
                                  evaluation_strategy="epoch",
                                logging_strategy="epoch",
                                  disable_tqdm=False,
                                  log_level="error",
                                 report_to="wandb")
#                                  optim="adafactor",) # saves memory bas a worse optimizer 

In [22]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [23]:
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=adverse_encoded["train"],
                  eval_dataset=adverse_encoded["valid"],
                  tokenizer=tokz)
trainer.train();  



Epoch,Training Loss,Validation Loss,Accuracy,F1
1,3.161600,3.066269,0.135417,0.064082
2,2.953500,2.753983,0.354167,0.233847
3,2.623600,2.437980,0.520833,0.437334
4,2.344800,2.151376,0.635417,0.566465
5,2.089800,1.907683,0.822917,0.789113
6,1.867800,1.718539,0.864583,0.829354
7,1.709900,1.576431,0.937500,0.929725
8,1.577700,1.473743,0.968750,0.965257
9,1.494000,1.414080,0.979167,0.978385
10,1.452200,1.393858,0.979167,0.978385


In [24]:
trainer.evaluate()

{'eval_loss': 1.3938575983047485,
 'eval_accuracy': 0.9791666666666666,
 'eval_f1': 0.9783854166666667,
 'eval_runtime': 1.6514,
 'eval_samples_per_second': 58.133,
 'eval_steps_per_second': 7.267,
 'epoch': 10.0}

In [25]:


# wandb.finish()



### Saving the Model and Loading it

In [27]:
# Save the model
path = '/content/drive/MyDrive/RESUME_YG/'
trainer.save_model(path + "resume_yg")



### Push the model to hugging face

In [28]:
# from huggingface_hub import notebook_login

In [29]:
# notebook_login()

In [30]:
# trainer.push_to_hub()

#### loading the model for inference

In [34]:
import datasets 
import optuna 
from datasets import load_dataset 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from os import path
import numpy as np
from numpy import argmax
path = '/content/drive/MyDrive/RESUME_YG/'

In [35]:
# # Load the model
# model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/fake_news_classifier/best_bert_model")

In [36]:
# Load the model
loaded_model = AutoModelForSequenceClassification.from_pretrained(
    path + "resume_yg",
    num_labels=25
)

# Load the tokenizer
loaded_tokenizer = AutoTokenizer.from_pretrained(
    path + "resume_yg",
)

# Max length
MAX_LENGTH = 512

In [38]:
example = df_test.inputs.iloc[0]
example

"Education Details \r\nJune 2014 to June 2018 BE Mechanical Engineering Pune, Maharashtra Savitribai Phule Pune University\r\nMechanical Design Engineer \r\n\r\nMechanical design engineer\r\nSkill Details \r\nMicrosoft office, Autocad, Catia, Solidworks- Exprience - 6 monthsCompany Details \r\ncompany - Push Engineering Pvt ltd\r\ndescription - Currently I am working as mechanical design engineer in Push engineering Pvt ltd. All types of ice making plant are designed and manufactured in our company. Design includes design of pressure vessel ,condenser, conveyor with the help of 3d modelling software. Also BOM Preparation of material.ERP System.\r\ncompany - Central Water and Power Research Station\r\ndescription - Pune)\r\n(Govt. of India)\r\n\r\n1 year experience as a Research assistant of R&D Department of mechanical workshop at\r\nCentral Water and Power Research Station, Khadakwasla, Pune (Govt. of india)\r\nâ\x80¢ Experiences different project of generation of sea waves in laborat

In [44]:


# Our example text to pass to our fine tuned model
text = example

def get_result(text, class_dict, message=True):
    encoded_input = loaded_tokenizer(text, truncation=True, padding='max_length',
                                     max_length=MAX_LENGTH, return_tensors='pt')
    output = loaded_model(**encoded_input)
    result = output[0].detach().numpy()
    probs = torch.sigmoid(output[0]).detach().numpy()
    class_label = np.argmax(result)
    label_name = class_dict[class_label]
    
    if message:
        result = f'The predicted career is label: {label_name} with a probability of {probs[0][class_label]}'
    
    return result



# Run your result through the function
get_result(text, class_dict)
     


'The predicted career is label: Mechanical Engineer with a probability of 0.8535858392715454'

### Perform evaluation on the whole held_out test set

In [45]:
# Perform predictions on the test dataset
preds = trainer.predict(adverse_encoded["test"])
preds


PredictionOutput(predictions=array([[ 7.9620391e-02,  2.4916457e-01,  1.1249562e-01, ...,
         2.4373510e-01,  1.9681120e-01, -1.6818875e-01],
       [-6.1664397e-01, -1.4783259e-01, -3.0605522e-01, ...,
        -3.6100143e-01, -6.6355759e-01, -1.3492529e-01],
       [-2.2100355e-01, -7.9403862e-02, -1.3649434e-01, ...,
         1.1807518e-01, -2.0590749e-01, -1.3589804e-01],
       ...,
       [-4.0158024e-01, -5.5968767e-01,  1.8473746e-01, ...,
        -4.6688676e-01, -2.9100493e-01, -2.0166117e-01],
       [-5.9323406e-01, -1.8722463e-01, -2.6992279e-01, ...,
        -2.6475769e-01, -5.3069448e-01, -1.9653791e-01],
       [-3.6661533e-01, -3.7282306e-01, -2.8598005e-01, ...,
        -3.2641202e-01, -4.8723835e-01,  6.3806109e-04]], dtype=float32), label_ids=array([16, 15,  5, 24, 20,  2, 19,  8,  0,  8, 10, 15, 22, 10, 12, 10, 21,
       15, 13, 23,  8,  3, 23,  3, 12, 21,  1,  7, 12, 20, 20, 19, 23,  9,
       11, 24, 18,  3, 18,  6,  6, 12,  9, 16, 24, 22,  2, 16,  7,  6, 11,

In [46]:
trainer.evaluate(adverse_encoded['test'])

{'eval_loss': 1.3424046039581299,
 'eval_accuracy': 0.9587628865979382,
 'eval_f1': 0.9555877157749105,
 'eval_runtime': 1.7319,
 'eval_samples_per_second': 56.008,
 'eval_steps_per_second': 7.506,
 'epoch': 10.0}

Evaluation accuracy and f1 of 96% wow

In [47]:
wandb.finish()

eval/accuracy,▁▃▄▅▇▇██████
eval/f1,▁▂▄▅▇▇██████
eval/loss,█▇▅▄▃▃▂▂▁▁▁▁
eval/runtime,▃▃▁▁█▄▃▄▂▃▂▇
eval/samples_per_second,▆▆██▁▄▆▅▇▆▇▃
eval/steps_per_second,▅▅▆▆▁▄▅▄▅▅▅█
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇█████
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇█████
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▇▆▅▄▃▂▂▁▁
train/total_flos,▁
